In [1]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D, AveragePooling2D,Activation, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import keras_tuner as kt

import rasterio
from rasterio.plot import reshape_as_image

from sklearn.model_selection import train_test_split
from skimage.transform import resize

width = 32 #32#83  --16
height= 32 #32#83  --16
layers = 7 #7 sat #6 drone
class_count = 5 #5
input_shape = (width, height, layers)
batch_size = 25 #25
epochs = 200

In [2]:
print(tf.__version__)
print(tf.config.list_physical_devices())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


2.7.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1


In [3]:
#simple CSV loader
def load_samples(csv_file):
    data = pd.read_csv(os.path.join(csv_file))
    data = data[['FileName','Label','ClassName']]
    file_names = list(data.iloc[:,0])
    #Get labels withing second column
    labels = list(data.iloc[:,1])
    samples=[]
    for samp,lab in zip(file_names,labels):
        samples.append([samp,lab])
    return samples

#ensure size of tile is uniform
def preprocessing(tile,label,class_count,layers,width,height):
    #print(tile.shape,"before")
    #to avoid artifacts, no antialiasing whn rescaling
    tile = resize(tile, (layers,width,height),anti_aliasing=False)
    #print(tile.shape,"resize")
    #Returns the source array reshaped into the order expected by image processing and visualization software (matplotlib, scikit-image, etc) by swapping the axes order from (bands, rows, columns) to (rows, columns, bands)
    tile = reshape_as_image(tile)
    #print(tile.shape,"reshape")
    
    #normalising tile
    tile = tile/255
    
    label = to_categorical(label,class_count)
    return tile,label

#Keras compatible data generator - works with rasterio compatible files (tiff only as written)
def generator(samples,batch_size,width,height,layers,class_count):
    """
    Yields next training batch, checks shape of tile, ensure image format - includes DEM
    """
    num_samples = len(samples)
    while True: # Loop forever so the generator never terminates
        #shuffle(samples)
        # Get index to start each batch: [0, batch_size, 2*batch_size, ..., max multiple of batch_size <= num_samples]
        for offset in range(0, num_samples, batch_size):
            # Get the samples you'll use in this batch
            batch_samples = samples[offset:offset+batch_size]

            # Initialise X_train and y_train arrays for this batch
            X_train = []
            y_train = []

            # For each example
            for batch_sample in batch_samples:
                # Load image (X) and label (y)
                img_name = batch_sample[0]
                label = batch_sample[1]
                #load in file
                #print(os.path.join(data_path,img_name))
                with rasterio.open(os.path.join(img_name)) as ds:
                    tile=ds.read()
                #perform any preprocessing required
                tile,label = preprocessing(tile,label,class_count,layers,width,height)     

                # Add example to arrays
                X_train.append(tile)
                y_train.append(label)

            # Make sure they're numpy arrays (as opposed to lists)
            X_train = np.array(X_train)
            #X_train = np.asarray(X_train).astype('float32')
            y_train = np.array(y_train)

            # The generator-y part: yield the next training batch            
            yield X_train, y_train

In [4]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            # Define the hyperparameter.
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            activation="relu",
        )
    )
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"],
    )
    return model

In [5]:
def Basic_CNN(input_shape,class_count,units,activation, dropout ,lr):
    #drop1 = dropout_rate, drop2 = dropout_rate_in
    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation(activation = activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if dropout:
        model.add(Dropout(0.5)) #0.25

    model.add(Flatten())
    model.add(Dense(units = units))
    model.add(BatchNormalization())
    model.add(Activation(activation = activation))
    if dropout:
        model.add(Dropout(0.5)) #0.25

    model.add(Dense(class_count))
    model.add(Activation(activation='softmax'))

    #model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=metrics)
    #opt = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate=lr),
                  metrics=['accuracy'])
    #print(model.summary())
    #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    
    return model

In [6]:
def build_model(hp):
    units = hp.Int("units", min_value=32, max_value=512, step=32)
    activation = hp.Choice("activation", ["relu", "tanh"])
    dropout = hp.Boolean("dropout")
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    # call existing model-building code with the hyperparameter values.
    model = Basic_CNN(
        input_shape =input_shape,units=units, activation=activation, dropout=dropout, lr=lr,class_count = class_count
    )
    return model

In [7]:
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=3,
    overwrite=True,
    directory=r"C:\Data\tune",
    project_name="BasicCNN_LULC_hyperparameter",
)

tuner.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [8]:

build_model(kt.HyperParameters())

data_path = 'C:\Data\Dataset'
data_dir_list = os.listdir(data_path)
print ('the data list is: ',data_dir_list)
fulldataset_df = load_samples('Full_drone_dataset.csv')

train_df,test_df = train_test_split(fulldataset_df, test_size=0.2) #0.2
test_df,valid_df = train_test_split(test_df, test_size=0.5)

#counts necessary
valid_count = len(valid_df)
train_count = len(train_df)
test_count = len(test_df)
print(valid_count)
print(train_count)
print(test_count)

#Create the generators 
train_datagen = generator(samples=train_df,batch_size=batch_size,width=width,height=height,layers=layers,class_count=class_count)
test_datagen = generator(samples=test_df,batch_size=batch_size,width=width,height=height,layers=layers,class_count=class_count)
valid_datagen = generator(samples=valid_df,batch_size=batch_size,width=width,height=height,layers=layers,class_count=class_count)

callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model-inprogress-tuning.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

#history = model.fit(train_datagen,steps_per_epoch=train_count // batch_size, verbose=1, epochs=epochs,validation_data=valid_datagen,validation_steps=valid_count // batch_size,callbacks=callbacks)
with tf.device('/GPU:0'):
    tuner.search(train_datagen,steps_per_epoch=train_count // batch_size,validation_steps=valid_count // batch_size, epochs=epochs, validation_data=valid_datagen,callbacks=callbacks,verbose=1)


Trial 1 Complete [11h 08m 19s]
val_accuracy: 0.9702057639757792

Best val_accuracy So Far: 0.9702057639757792
Total elapsed time: 11h 08m 19s

Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
units             |32                |480               
activation        |tanh              |relu              
dropout           |True              |True              
lr                |0.00021478        |0.00011307        

Epoch 1/200
1300/1300 [==============================] - ETA: 0s - loss: 0.6732 - accuracy: 0.7578
Epoch 00001: val_loss improved from inf to 1.69763, saving model to model-inprogress-tuning.h5
1300/1300 [==============================] - 329s 253ms/step - loss: 0.6732 - accuracy: 0.7578 - val_loss: 1.6976 - val_accuracy: 0.5669 - lr: 2.1478e-04
Epoch 2/200
1300/1300 [==============================] - ETA: 0s - loss: 0.4432 - accuracy: 0.8476
Epoch 00002: val_loss improved from 1.69763 to 1.00581, saving model to model-inprogress-tuning.h5

KeyboardInterrupt: 

In [9]:
tuner.results_summary()

Results summary
Results in C:\Data\tune\BasicCNN_LULC_hyperparameter
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
units: 480
activation: relu
dropout: True
lr: 0.0001130694221649183
Score: 0.9702057639757792


In [10]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
#best_model.build(input_shape=(None, 28, 28))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        4096      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 64)        0         
                                                                 
 flatten (Flatten)           (None, 16384)             0

In [13]:
    #save model for later use
best_model.save('BasicCNN_LULC_tuned')

INFO:tensorflow:Assets written to: BasicCNN_LULC_tuned\assets
